In [1]:
import sys
sys.path.append('../')

In [2]:
%set_env PYTHONPATH ../

env: PYTHONPATH=../


In [3]:
import ltorch
from ltorch.apps.models import ResNet9
from ltorch.apps.simple_ml import train_cifar10, evaluate_cifar10
import numpy as np

device = ltorch.cuda()
dataset = ltorch.data.CIFAR10Dataset("ltorch_data/cifar-10-batches-py", train=True)
dataloader = ltorch.data.DataLoader(
         dataset=dataset,
         batch_size=256,
         shuffle=True, device=device)
model = ResNet9(device=device, dtype="float32")

loss_fn = ltorch.nn.SoftmaxLoss()
n_epochs=50
optimizer=ltorch.optim.Adam
lr=0.001
weight_decay=0.001

opt = optimizer(model.parameters(), lr=lr, weight_decay=weight_decay)
for epoch in range(n_epochs):
    correct, total_loss = 0, 0
    device = model.device
    model.train()
    for batch in dataloader:
        opt.reset_grad()
        X, y = batch
        X, y = ltorch.Tensor(X, device=device), ltorch.Tensor(y, device=device)
        out = model(X)
        loss = loss_fn(out, y)
        loss.backward()
        opt.step()
        correct += np.sum(np.argmax(out.numpy(), axis=1) == y.numpy())
        total_loss += loss.data.numpy() * y.shape[0]
    sample_nums = len(dataloader.dataset)
    avg_acc, avg_loss =  correct / sample_nums, total_loss / sample_nums
    print(f"Epoch: {epoch}, Acc: {avg_acc}, Loss: {avg_loss}")

evaluate_cifar10(model, dataloader)


Using ltorch backend
Epoch: 0, Acc: 0.36926, Loss: [1.7610228]
Epoch: 1, Acc: 0.47838, Loss: [1.4424644]
Epoch: 2, Acc: 0.52232, Loss: [1.3144271]
Epoch: 3, Acc: 0.55766, Loss: [1.223388]
Epoch: 4, Acc: 0.58552, Loss: [1.1526308]
Epoch: 5, Acc: 0.60802, Loss: [1.0963652]
Epoch: 6, Acc: 0.62528, Loss: [1.0428902]
Epoch: 7, Acc: 0.6457, Loss: [0.9904319]
Epoch: 8, Acc: 0.6629, Loss: [0.94632685]
Epoch: 9, Acc: 0.67764, Loss: [0.9094294]
Epoch: 10, Acc: 0.68838, Loss: [0.86830944]
Epoch: 11, Acc: 0.70258, Loss: [0.8285485]
Epoch: 12, Acc: 0.71576, Loss: [0.79799104]
Epoch: 13, Acc: 0.72844, Loss: [0.76214737]
Epoch: 14, Acc: 0.73786, Loss: [0.73106116]
Epoch: 15, Acc: 0.74862, Loss: [0.702864]
Epoch: 16, Acc: 0.76058, Loss: [0.6702304]
Epoch: 17, Acc: 0.76808, Loss: [0.64752674]
Epoch: 18, Acc: 0.77886, Loss: [0.6195581]
Epoch: 19, Acc: 0.7912, Loss: [0.588093]
Epoch: 20, Acc: 0.796, Loss: [0.57328856]
Epoch: 21, Acc: 0.8043, Loss: [0.5489439]
Epoch: 22, Acc: 0.80972, Loss: [0.52895224]
E

In [4]:
import numpy as np
test_dataset = ltorch.data.CIFAR10Dataset("ltorch_data/cifar-10-batches-py", train=False)
test_dataloader = ltorch.data.DataLoader(\
         dataset=test_dataset,
         batch_size=1,
         shuffle=False,)


model.eval()

step = 0
for batch in test_dataloader:
    step += 1
    if step > 10:
        break
    image = batch[0]
    label = batch[1]
    x, y = ltorch.Tensor(image, device=device), ltorch.Tensor(label, device=device)
    pred = model(x)
    print("label: ", y, "pred: ", np.argmax(pred.numpy(), axis=1))

label:  [3.] pred:  [3]
label:  [8.] pred:  [8]
label:  [8.] pred:  [8]
label:  [0.] pred:  [8]
label:  [6.] pred:  [6]
label:  [6.] pred:  [6]
label:  [1.] pred:  [3]
label:  [6.] pred:  [6]
label:  [3.] pred:  [3]
label:  [1.] pred:  [1]
